# Monitoring the Model

When you've deployed a model into production as a service, you'll want to monitor it to track usage and explore the requests it processes. In this lab, you'll use Azure Application Insights to monitor activity for a model service endpoint.

## Connect to Your Workspace

The first thing you need to do is to connect to your workspace using the Azure ML SDK.

> **Note**: You may be prompted to authenticate. Just copy the code and click the link provided to sign into your Azure subscription, and then return to this notebook.

In [ ]:
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

### Enable Application Insights

Next, you need to enable Application Insights for the service.

In [ ]:

from azureml.core.webservice import Webservice

# Enable AppInsights
aks_service = Webservice(ws, 'flight-service')
aks_service.update(enable_app_insights=True)
print(aks_service.state)
print('AppInsights enabled!')

## Use the Web Service

With the service deployed and application insight enabled now you can consume it from a client application.

In [ ]:
import requests
import json

endpoint = 'http://137.135.12.229:80/api/v1/service/flight-service/score'
primary_key = 'SODNH29sWD8tmyjDdJgFlPGgdvoT6Xy6'

x_new = [[4, 19, 5, 4, 18, 36, 837, -3.0, 1138],
         [3, 76, 6, 4, 19, 36, 837, 98.0, 1234]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
request_headers = {"Content-Type": "application/json",
                   "Authorization": "Bearer " + primary_key}

predictions = requests.post(endpoint, input_json, headers=request_headers)

predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print("Flight {}".format(x_new[i]), predicted_classes[i])


Now you can view the data logged for the service endpoint:
1. In the [Azure portal](https://portal.azure.com), open your Machine Learning workspace.
2. On the **Overview** page, click the link for the associated **Application Insights** resource.
3. On the Application Insights blade, click **Logs**. 

    > **Note**: If this is the first time you've opened log analytics, you may need to click **Get Started** to open the query editor. If a tip explaining how to write a query is displayed, close it.

4. Paste the following query into the query editor and click **Run**
    ```
    traces
    |where  message == "STDOUT"
      and customDimensions.["Service Name"] == "flight-service"
    |project timestamp, customDimensions.Content
    ```
5. View the results. At first there may be none, because an AKS web service can take two to three minutes to send the telemetry to Application Insights. Wait a few minutes and re-run the query until you see the logged data and predictions.